In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='TRUE'

import warnings
# 忽略所有的 featurewarnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import SubprocVecEnv
import torch

env_case_ids = [6, 13, 2617, 148, 809]
envs = SubprocVecEnv(
    [
        lambda: torch.load("./temp_envs/env_{}".format(case_id), weights_only=False)
        for case_id in env_case_ids
    ]
)
obs = envs.reset()
print(f"obs shape: {obs.shape}")

# envs = RecordVideo(envs, "./videos", episode_trigger=lambda e: True)

model = PPO.load(r"./models/multi/PPO_20250327011246.zip")
obs, info = envs.reset()
for _ in range(1000):
    actions, _ = model.predict(obs)
    print(f"actions: {actions}")
    obs, reward, terminal, truncated, info = envs.step(actions)
    envs.render()
    print(f"info: {info}")
    if info["crashed"] or info["offroad"]:
        print(f"crashed: {info['crashed']}, offroad: {info['offroad']}")
        break

    
envs.close()